# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
import json

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files("data/song_data")

In [5]:
df_all=pd.DataFrame()
df =df_all
for songfile in song_files:
    filepath = songfile
    df_all = pd.read_json(filepath,lines=True)
    df=df_all[['song_id','title','artist_id','year','duration']]
    song_data = df.values[0].tolist()
    cur.execute(song_table_insert, song_data)
    conn.commit()
    #Artist of the album
    artist_data = df_all[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values[0].tolist()
    cur.execute(artist_table_insert, artist_data)
    conn.commit()

In [6]:
#filepath = song_files[0]

In [7]:

#df_all = pd.read_json(filepath,lines=True)
df_all.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARMJAGH1187FB546F3,35.14968,"Memphis, TN",-90.04892,The Box Tops,148.03546,1,SOCIWDW12A8C13D406,Soul Deep,1969


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
#df = df_all[['song_id','title','artist_id','year','duration']]
df.values

array([['SOCIWDW12A8C13D406', 'Soul Deep', 'ARMJAGH1187FB546F3', 1969,
        148.03546]], dtype=object)

In [9]:
#song_data = df.values[0].tolist()
#song_data 

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [10]:
print(song_table_insert,song_data)

insert into songs(song_id, title, artist_id, year, duration)                       values(%s,%s,%s,%s,%s); ['SOCIWDW12A8C13D406', 'Soul Deep', 'ARMJAGH1187FB546F3', 1969, 148.03546]


In [11]:
#cur.execute(song_table_insert, song_data)
#conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [12]:
df_all[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values

array([['ARMJAGH1187FB546F3', 'The Box Tops', 'Memphis, TN', 35.14968,
        -90.04892]], dtype=object)

In [13]:
artist_data = df_all[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values[0].tolist()
artist_data

['ARMJAGH1187FB546F3', 'The Box Tops', 'Memphis, TN', 35.14968, -90.04892]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [14]:
#cur.execute(artist_table_insert, artist_data)
#conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [15]:
log_files = get_files("data/log_data")

In [16]:
filepath = log_files[0]

In [17]:
df = pd.read_json(filepath,lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


In [18]:
df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [19]:
len(df)

366

In [20]:
df = df[df.page.str.contains('NextSong')]
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


In [21]:
len(df)

319

In [22]:
import pandas as pd
from datetime import datetime
t =pd.to_datetime(df['ts']) 
#t

In [23]:
time_data = t.dt.to_pydatetime()
column_labels = ('year','month','day','hour','weekday','week','millis')
time_data[1]

datetime.datetime(1970, 1, 1, 0, 25, 43, 449690)

In [24]:
from datetime import date
time_df = pd.DataFrame(columns=[col for col in column_labels])
time_df['year'] = t.dt.year
time_df['month'] = t.dt.month
time_df['day']= t.dt.day
time_df['hour']= t.dt.hour
time_df['weekday']= t.dt.weekday
time_df['week']= t.dt.week
time_df['millis']= t.dt.time


In [25]:
#time_df = pd.DataFrame(data=time_data,columns=column_labels)
time_df.head()

,year,month,day,hour,weekday,week,millis
0,1970,1,1,0,3,1,00:25:43.449657
1,1970,1,1,0,3,1,00:25:43.449690
2,1970,1,1,0,3,1,00:25:43.449841
3,1970,1,1,0,3,1,00:25:43.449895
4,1970,1,1,0,3,1,00:25:43.450033


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [26]:
for i, row in time_df.iterrows():
    print(list(row))
    try:
        cur.execute(time_table_insert, list(row))
        conn.commit()
    except psycopg2.Error as e: 
        print("Error: Could not make connection to the Postgres database")
        print(e)
                    

[1970, 1, 1, 0, 3, 1, datetime.time(0, 25, 43, 449657)]
[1970, 1, 1, 0, 3, 1, datetime.time(0, 25, 43, 449690)]
[1970, 1, 1, 0, 3, 1, datetime.time(0, 25, 43, 449841)]
[1970, 1, 1, 0, 3, 1, datetime.time(0, 25, 43, 449895)]
[1970, 1, 1, 0, 3, 1, datetime.time(0, 25, 43, 450033)]
[1970, 1, 1, 0, 3, 1, datetime.time(0, 25, 43, 450256)]
[1970, 1, 1, 0, 3, 1, datetime.time(0, 25, 43, 450264)]
[1970, 1, 1, 0, 3, 1, datetime.time(0, 25, 43, 450450)]
[1970, 1, 1, 0, 3, 1, datetime.time(0, 25, 43, 450562)]
[1970, 1, 1, 0, 3, 1, datetime.time(0, 25, 43, 450691)]
[1970, 1, 1, 0, 3, 1, datetime.time(0, 25, 43, 450933)]
[1970, 1, 1, 0, 3, 1, datetime.time(0, 25, 43, 451175)]
[1970, 1, 1, 0, 3, 1, datetime.time(0, 25, 43, 451453)]
[1970, 1, 1, 0, 3, 1, datetime.time(0, 25, 43, 451707)]
[1970, 1, 1, 0, 3, 1, datetime.time(0, 25, 43, 452014)]
[1970, 1, 1, 0, 3, 1, datetime.time(0, 25, 43, 452253)]
[1970, 1, 1, 0, 3, 1, datetime.time(0, 25, 43, 452394)]
[1970, 1, 1, 0, 3, 1, datetime.time(0, 25, 43, 4

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [27]:
user_df = df[['userId','firstName','lastName','gender','level']]

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [28]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [29]:
df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

In [30]:
from datetime import datetime
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    print((row.song, row.artist, row.length))
    try:
        cur.execute(song_select, (row.song, row.artist, row.length))
        results = cur.fetchone()
    except psycopg2.Error as e: 
        print("Error: Could not get song and artist data from the Postgres database")
        print(e)
    
    if results:
        songid, artistid = results
        print(results)
    else:
        songid, artistid = None, None

    # insert songplay record
    #tm=(datetime.fromtimestamp(row.ts/1000))
    print((datetime.fromtimestamp(row.ts/1000)).time())
    #tm= (datetime.fromtimestamp(row.ts/1000)).dt.time
    songplay_data = ((datetime.fromtimestamp(row.ts/1000)).time(),row.userId,row.level,songid,artistid,row.sessionId,row.location,row.userAgent)
    
    try:
        print(songplay_table_insert,songplay_data)
        cur.execute(songplay_table_insert, songplay_data)
        conn.commit()
    except psycopg2.Error as e: 
        print("Error: Could not make connection to the Postgres database")
        print(e)

("Ain't No Sunshine", 'Sydney Youngblood', 238.07955)
00:00:57.796000
insert into songplays(start_time,user_id,level,song_id,                          artist_id,session_id,location,user_agent)                          values(%s,%s,%s,%s,%s,%s,%s,%s); (datetime.time(0, 0, 57, 796000), '73', 'paid', None, None, 954, 'Tampa-St. Petersburg-Clearwater, FL', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"')
('My Advice 2 You (Explicit)', 'Gang Starr', 151.92771)
00:01:30.796000
insert into songplays(start_time,user_id,level,song_id,                          artist_id,session_id,location,user_agent)                          values(%s,%s,%s,%s,%s,%s,%s,%s); (datetime.time(0, 1, 30, 796000), '24', 'paid', None, None, 984, 'Lake Havasu City-Kingman, AZ', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"')
('My First Kiss (Feat. Ke$ha) [Album Version]', '3OH!3', 

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [32]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.